<h1> Sharwari Dali

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings 
import plotly.offline as py
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/gtd/globalterrorismdb_0718dist.csv')
df.head(10) #displaying head values

In [ ]:
df.tail()  #display the tail values

In [ ]:
#Renaming the columns for ease of using them later
df.rename(columns={'iyear':'Year','imonth':'Month','iday':'Day','country_txt':'Country','provstate':'state',
                     'region_txt':'Region','attacktype1_txt':'AttackType','target1':'Target','nkill':'Killed',
                     'nwound': 'Wounded','summary':'Summary','gname':'Group','targtype1_txt':'Target_type',
                     'weaptype1_txt':'Weapon_type','motive':'Motive'
                    },inplace=True)

In [ ]:
#getting only the columns to be used
df = df[['Year','Month','Day','Country','Region','state','city','latitude','longitude','AttackType','Killed','Wounded','Target',
            'Target','Summary','Group','Target_type','Weapon_type','Motive']]

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

<h3> Most Attacked 

In [ ]:
print("Most Attacked")
print("Country : ",df['Country'].value_counts().idxmax())
print('Region :',df['Region'].value_counts().idxmax())
print('Year : ',df['Year'].value_counts().idxmax())
print('Month : ',df['Month'].value_counts().idxmax())
print('Type: ',df['AttackType'].value_counts().idxmax())


In [ ]:
plt.subplots(figsize=(15,6))
sns.barplot(df['Country'].value_counts()[:15].index,df['Country'].value_counts()[:15].values,palette='Blues_d')
plt.title('Top Countries Affected',fontsize=16)
plt.xlabel('Countries')
plt.ylabel('Count')
plt.xticks(rotation= 90)
plt.show()

Conclusion: Iraq is the most affected Country

<h3> Number of Attacks as per year

In [ ]:
x_year = df['Year'].unique()
y_count_year = df['Year'].value_counts(dropna=True).sort_index()
plt.figure(figsize=(15,8))
sns.barplot(x=x_year,y=y_count_year, palette='PuBu')
plt.xticks(rotation = 45)
plt.xlabel('Attack Year')
plt.ylabel('Number of Attacks per year')
plt.title('Attack every year',fontsize=16)
plt.show()

Conclusion: Since 2014 the attacks have been more compared to previous years

In [ ]:
pd.crosstab(df['Year'], df['Region']).plot(kind='area', figsize=(20,8))
plt.xlabel('Year')
plt.ylabel('Number of Attcks')

Conclusion: South Asia Region had more attacks during and after 2010

<h3> Casualities

In [ ]:
df['Wounded'] = df['Wounded'].fillna(0).astype(int)
df['Killed'] = df['Killed'].fillna(0).astype(int)
df['casualities'] = df['Wounded']+df['Killed']

In [ ]:
df = df.sort_values(by='casualities')[:]

In [ ]:
heat = df.pivot_table(index = 'Country',columns='Year',values = 'casualities')
heat.fillna(0,inplace=True)
print(heat.head())

In [ ]:
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
colorscale = [[0, '#edf8fb'], [.3, '#00BFFF'],  [.6, '#8856a7'],  [1, '#810f7c']]
heatmap = go.Heatmap(z=heat.values, x=heat.columns, y=heat.index, colorscale=colorscale)
data2 = [heatmap]
l = go.Layout(xaxis=dict(ticks='', nticks=20),  yaxis = dict(ticks=''))
fig = go.Figure(data=data2, layout=l)
py.iplot(fig, filename='heatmap',show_link=False)

In [ ]:
import folium
from folium.plugins import MarkerCluster
filteryear = df['Year']==1970
filterData = df[filteryear]
reqfiltData = filterData.loc[:,'city':'longitude']
reqfiltData = reqfiltData.dropna()
reqDatalist = reqfiltData.values.tolist()

In [ ]:
map = folium.Map(location=[0,30],tiles = 'CartoDB positron', zoom_start=3)
markercluster = folium.plugins.MarkerCluster().add_to(map)
for point in range(0,len(reqDatalist)):
    folium.Marker(location = [reqDatalist[point][1],reqDatalist[point][2]], popup= reqDatalist[point][0]).add_to(markercluster)
map

In [ ]:
df.Group.value_counts()[1:15]

In [ ]:
unq_country = df[df.Group.isin(['Shining Path (SL)','Taliban','Islamic State of Iraq and the Levant (ISIL'])]

In [ ]:
unq_country.Country.unique()

In [ ]:
Killed = df.loc[:,'Killed']
print('Number of people killed by terror attack:',int(sum(Killed.dropna())))

In [ ]:
typeKilledData = df.pivot_table(columns='AttackType',values = 'Killed',aggfunc = 'sum')
typeKilledData

In [ ]:
countrydata = df.loc[:,'Country'] 
countrykilled = pd.concat([countrydata,Killed],axis=1)
countrykilled = countrykilled.pivot_table(columns = 'Country', values='Killed',aggfunc='sum')
countrykilled

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 25
fig_size[1] = 25
plt.rcParams["figure.figsize"] = fig_size

In [ ]:
labels = countrykilled.columns.tolist()
labels = labels[:50]
index = np.arange(len(labels))
transpose = countrykilled.T
values = transpose.values.tolist()
values = values[:50]
values = [int(i[0]) for i in values]
colors = ['red','green','purple','yellow','brown','black','gray','magenta','orange']
fig,ax = plt.subplots(1,1)
ax.yaxis.grid(True)
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 25
fig_size[1] = 25
plt.rcParams["figure.figsize"] = fig_size
plt.bar(index,values,color = colors,width=0.9)
plt.xlabel('Countries',fontsize=20)
plt.ylabel('Killed People',fontsize = 20)
plt.xticks(index,labels,fontsize=18,rotation=90)
plt.title('Number of people killed by countries',fontsize = 20)
plt.show()

Conclusion: Afghanistan lost the most lives during the attack

<h3> Conclusions:

* The Hot Zones of Terrorism are Iraq, Afghanistan, Colombia
* As a huge number of People are killed by the terror attack, We need to take strict actions to demolish groups like Taliban, SL etc